In [ ]:
# A very simple Flask Hello World app for you to get started with...

from flask import Flask, render_template, url_for, redirect,flash, request, session
from scipy.io import wavfile
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import decimate
from keras.models import load_model
import pandas as pd

app = Flask(__name__)
app.config['UPLOAD_FOLDER']='/home/HeartSound/uploads'
@app.route('/')
def home_page():
    return render_template('main.html')


@app.route('/predict/', methods=["POST"])
def predict():
    try:

        file = request.files['file']
        print (file)
        filename = file.filename
        print (filename)
        print (app.config['UPLOAD_FOLDER'])
        print (file.save(os.path.join(app.config['UPLOAD_FOLDER'],filename)))
        file.stream.seek(0)
        myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)

        [fs, x]=wavfile.read(myfile)


        #%matplotlib inline
        model = load_model('/home/HeartSound/mysite/my_model.h5')
        MAX_LEN = 396900

        def repeat_to_length(array, length):
                ## Repeat the numpy 1D array to given length, and makes datatype float
                result = np.empty((length, ), dtype = 'float32')
                l = len(array)
                pos = 0
                if l > length:
                   result = array[:length]
                while pos + l <= length:
                   result[pos:pos+l] = array # repeat the whole array
                   pos += l
                if pos < length:
                   result[pos:length] = array[:length-pos] # repeat the beginning part
                return result
        x = repeat_to_length(x, length=MAX_LEN) # make all the data same length (max length)

        x = decimate(x, 8, zero_phase=True)
        x = decimate(x, 8, zero_phase=True)
        x = decimate(x, 4, zero_phase=True)

        x = x / np.std(x).reshape(-1,1)

        x = x[:,:,np.newaxis]

        y_pred = model.predict(x)

        a = np.argmax(y_pred)
        CLASSES = ['Artifact -.-', 'Normal ^.^', 'Murmur T^T']

        return render_template("main.html", result= (CLASSES[a]))

    except Exception as e:
        return str(e)